<a href="https://colab.research.google.com/github/t8101349/Colab-/blob/master/gradio_web_0318.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install numpy
!pip install gradio
!pip install rdkit
!pip install scikit-learn
!pip install xgboost
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [ ]:
import gradio as gr
import pandas as pd
import traceback
from gradio.themes import Base
from process import predict_process

class AppState:
    def __init__(self):
        self.df = None
        self.result_df = None
        self.file_uploaded = False
        self.prediction_done = False

state = AppState()

def confirm_file(file):
    if file is None:
        return "請上傳分子數據集！", False
    try:
        original_filename = file.name if hasattr(file, "name") else "未知檔案"
        if original_filename.endswith('.csv'):
            df = pd.read_csv(file.name)
        elif original_filename.endswith('.parquet'):
            df = pd.read_parquet(file.name)
        else:
            return f"不支援的檔案格式：{original_filename}！僅支援 .csv 和 .parquet", False
        required_columns = ["molecule_smiles", "protein_name"]
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            return f"檔案缺少必要的欄位：{', '.join(missing_columns)}", False
        if len(df) > 500000:
            return "資料筆數超過 50 萬筆，請減少資料量！", False
        state.df = df
        state.file_uploaded = True
        state.prediction_done = False
        return f"✅ 已成功上傳檔案：{original_filename}，共 {len(df)} 筆資料", True
    except Exception as e:
        error_details = traceback.format_exc()
        print(f"檔案處理錯誤：{str(e)}\n{error_details}")
        return f"❌ 檔案處理錯誤：{str(e)}", False

def run_prediction():
    if not state.file_uploaded or state.df is None:
        return "❌ 請先上傳並確認檔案！", False
    try:
        result_df = predict_process(state.df)
        state.result_df = result_df
        state.prediction_done = True
        return f"✅ 預測完成！共處理 {len(result_df)} 筆資料", True
    except Exception as e:
        error_details = traceback.format_exc()
        print(f"預測錯誤：{str(e)}\n{error_details}")
        return f"❌ 預測錯誤：{str(e)}", False

def generate_file(format_choice):
    import tempfile
    import os
    if not state.prediction_done or state.result_df is None:
        return None, "❌ 請先執行預測！"
    try:
        temp_dir = tempfile.gettempdir()
        if format_choice == "CSV":
            filename = "prediction.csv"
            filepath = os.path.join(temp_dir, filename)
            state.result_df.to_csv(filepath, index=False)
        else:
            filename = "prediction.parquet"
            filepath = os.path.join(temp_dir, filename)
            state.result_df.to_parquet(filepath, index=False)
        return filepath, f"✅ 已生成 {filename}，點擊下方按鈕即可下載"
    except Exception as e:
        error_details = traceback.format_exc()
        print(f"生成檔案錯誤：{str(e)}\n{error_details}")
        return None, f"❌ 生成檔案錯誤：{str(e)}"

def update_button_status(status_text, button_value):
    if "✅" in status_text:
        return gr.update(interactive=True, value="執行預測")
    else:
        return gr.update(interactive=False, value="執行預測")

custom_theme = Base(
    primary_hue="cyan",
    secondary_hue="teal",
    neutral_hue="gray"
)

with gr.Blocks(
    theme=custom_theme,
    title="新藥預測工具",
    css="""
    .gradio-container {
        width: 800px !important;
        margin: auto !important;
    }
    h2 {
        text-align: center;
        font-size: 20px;
    }
    .gradient-title h1 {
        background: linear-gradient(45deg, #13A9E6, #3DD69E);
        -webkit-background-clip: text;
        background-clip: text;
        color: transparent;
        text-align: center;
        font-size: 52px;
        font-weight: bold;
    }
    .gr-radio input[type="radio"] {
        accent-color: #13A9E6;
    }
    footer {
        display: none !important;
    }
    .file-status {
        font-weight: bold;
    }
    """
) as demo:
    gr.Markdown("# 新藥預測工具", elem_classes=["gradient-title"])

    with gr.Accordion("點此查看工具詳細說明", open=False):
        gr.Markdown("""
            **詳細說明：**
            此工具可將SMILES形式的分子資料集根據一個預測分子與三種蛋白質標靶(sEH, BRD4, HSA)
            是否結合的機器學習模型，來快速篩選出可能的藥物分子資料集。
            **操作說明：**
            1. 上傳分子數據集 (支援 CSV 與 Parquet 格式, 檔案大小上限50MB, 資料筆數上限50萬筆)
            2. 確認上傳檔案
            3. 執行預測
            4. 選擇下載格式
            5. 產生並下載預測檔案
            **必要欄位說明：**
            - molecule_smiles: 分子的SMILES表示法
            - protein_name: 蛋白質名稱 (必須為 sEH, BRD4, HSA 其中之一)
            """)

    with gr.Column():
        gr.Markdown("## 上傳分子數據集", elem_classes=["sub_title"])

        file_input = gr.File(
            label="拖曳檔案至此或點擊上傳，上限50MB",
            file_types=[".csv", ".parquet"],
            type="filepath"
        )

        confirm_btn = gr.Button("確認檔案", variant="primary")
        file_status = gr.Textbox(
            label="檔案狀態",
            elem_classes=["file-status"],
            interactive=False
        )

        predict_btn = gr.Button("執行預測", variant="primary", interactive=False)
        predict_status = gr.Textbox(
            label="預測狀態",
            elem_classes=["file-status"],
            interactive=False
        )

        with gr.Row():
            download_format = gr.Radio(
                choices=["CSV", "Parquet"],
                label="選擇下載格式",
                value="CSV"
            )

        generate_btn = gr.Button("產生下載檔案", variant="primary", interactive=False)
        generate_status = gr.Textbox(
            label="檔案生成狀態",
            elem_classes=["file-status"],
            interactive=False
        )

        download_btn = gr.DownloadButton(
            label="下載預測結果",
            variant="primary",
            interactive=False,
            visible=True
        )

    confirm_btn.click(
        fn=confirm_file,
        inputs=file_input,
        outputs=[file_status, predict_btn]
    ).then(
        fn=update_button_status,
        inputs=[file_status, predict_btn],
        outputs=predict_btn
    )

    predict_btn.click(
        fn=run_prediction,
        inputs=None,
        outputs=[predict_status, generate_btn]
    ).then(
        fn=lambda status_text, btn_value: gr.update(
            interactive="✅" in status_text,
            value="產生下載檔案"
        ),
        inputs=[predict_status, generate_btn],
        outputs=generate_btn
    )

    generate_btn.click(
        fn=generate_file,
        inputs=download_format,
        outputs=[download_btn, generate_status]
    ).then(
        fn=lambda filepath, status: (
            gr.update(
                value=filepath,
                interactive=True,
                visible=True
            ) if "✅" in status else gr.update(
                value=None,
                interactive=False,
                visible=True
            )
        ),
        inputs=[download_btn, generate_status],
        outputs=download_btn
    )

if __name__ == "__main__":
    demo.launch(
        # 設定上傳檔案大小50MB限制
        max_file_size=50 * 1024 * 1024
    )